# Experiment Introduction

Background, 
In order to do ablation experiment, we have diffculty like
1. most of them are same, we have to repeat many times.
2. too many model, config, code hard to manage and save. too messy


For all kind of SSL training workflow, we have to define the hyperparameters includes 4 aspect,
Dataset
Model
Training
Saving COnfig



How to Use this Experiment?



# Library

In [1]:
# Import config
import sys 
sys.path.append("C:/Users/isxzl/OneDrive/Code/VICReg-BarlowTwins-Ablation-Study/autoSSL")
sys.path.append("C:/Users/isxzl/OneDrive/Code/VICReg-BarlowTwins-Ablation-Study/")
import torch
import yaml
from torchvision.transforms import RandomRotation,GaussianBlur,ColorJitter
from autoSSL.evaluate import eval_KNN,eval_linear,eval_KNNplot,pipe_collate
from autoSSL.models import BarlowTwins, BYOL, MoCo, SimCLR, SimSiam, VICReg ,pipe_model 
from autoSSL.utils import embedding_feature,ck_callback,dict2transformer,join_dir,ContinuousCSVLogger  
from autoSSL.data import PipeDataset
from autoSSL.train import Trainer
 


C:\Users\isxzl\anaconda3\envs\AutoGPT\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Import Global Baseline Config

In [2]:
# Load the YAML file
with open('global.yaml', 'r') as file:
    global_config = yaml.safe_load(file)

# Write your experiment notebook name here
global_config["experiment"]="backbone VS model"     
    
# Define global view function
global_SSL_augmentation = {
    'RandomResizedCrop': {'size': (global_config["input_size"], global_config["input_size"])},
    'RandomApply':{'transforms':[RandomRotation(degrees=90)], 'p':0.8},
    'RandomHorizontalFlip': {'p': 0.5},
    'RandomVerticalFlip':  {'p':0.5},
    'RandomApply':{'transforms': [ColorJitter(brightness=0.04,contrast=0.04,saturation=0.02,hue=0.01)], 'p':0.8},
    'RandomGrayscale' :{'p':0.2},
    'RandomSolarize':{'threshold':128, 'p':0.1},
    'RandomApply':{'transforms':[GaussianBlur(kernel_size=3,sigma=(0.2, 2))],'p':0.8},
    'ToTensor': {},
    'Normalize': {"mean": [0.485, 0.456, 0.406], "std": [0.229, 0.224, 0.225]}
}


# Train Model

## vicreg

In [ ]:
 
del pdata
del pmodel
del trainer
torch.cuda.empty_cache()

In [ ]:
import GPUtil
GPUs = GPUtil.getGPUs()
gpu = GPUs[0]
import torch
print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))


In [ ]:
import torch
config=global_config.copy()
config["model"]="VICReg"
#["vit_64","efficientnet_b5", "efficientnet_b5_pretrained", "mobilenet_v3", "mobilenet_v3_pretrained"]:

config["backbone"]="efficientnet_b5"
#config["batch"]=512
#config["prjhead_dim"]=1024
pmodel=pipe_model(config=config)
image = torch.randn(32, 3, 64, 64)

from torch.profiler import profile, record_function

# Then, wrap the training loop or the part of your code you want to profile with the profiler context manager.
with torch.profiler.profile(activities=[torch.profiler.ProfilerActivity.CPU, torch.profiler.ProfilerActivity.CUDA], record_shapes=True) as prof:
    with record_function("model_inference"):
        output = pmodel(image)

# Finally, print out the profiling result.
print(prof.key_averages().table(sort_by="cuda_time_total"))

In [ ]:
# "resnet18", "resnet18_pretrained", "resnet50", "resnet50_pretrained", "vit_64",
for backbone_name in["mobilenet_v3", "mobilenet_v3_pretrained", "efficientnet_b5", "efficientnet_b5_pretrained"]:

    # MAKE YOUR OWN CONFIG
    config=global_config.copy()
    # Fill the config
    SSL_augmentation=global_SSL_augmentation.copy()
    config["name"]=f"VICReg_backbone_{backbone_name}"
    config["backbone"]=backbone_name
    config["model"]="VICReg"
    config["batch"]=512
    config["prjhead_dim"]=1024
    # THIS IS THE CODE TO LOAD DATASET
    pdata= PipeDataset(config=config,augmentation=dict2transformer(SSL_augmentation,view=config["view"]))
    # THIS IS THE CODE TO LOAD MODEL
    pmodel=pipe_model(config=config)

    # Use this if you want to START a train
    trainer=Trainer(config, model_mode="start")
    trainer.fit(pmodel, pdata.dataloader)  

    del pdata
    del pmodel
    del trainer
    torch.cuda.empty_cache()
    # Use this if you want to CONTINUE to train
    #trainer1=Trainer(config, model_mode="continue", extra_epoch=0)
    #trainer1.fit(pmodel, pdata.dataloader,ckpt_path="latest")  

## Config 2

## Config 3

# Evaluation

In [ ]:

test_augmentation = {
    'RandomResizedCrop': {'size': (global_config["input_size"], global_config["input_size"])},
    'ToTensor': {},
    'Normalize': {"mean": [0.485, 0.456, 0.406], "std": [0.229, 0.224, 0.225]}
}
collate =pipe_collate(address="experiment_checkpoints/backbone VS model/", reg="^VICReg_backbone_.*$")

pdata = PipeDataset(input_dir=global_config["path_to_test_cifar10"],
    augmentation=dict2transformer(test_augmentation,view=1))


In [12]:

aaa=eval_linear(pdata, models=collate, device=global_config["device"], split=0.8)


Load the training and testing dataset


 20%|██████████████▊                                                             | 1563/8000 [00:00<00:03, 1693.17it/s]


KeyboardInterrupt: 

In [11]:
collate["name"]= collate["name"][0:2]
collate["model"]= collate["model"][0:2]

In [14]:
collate["name"] 

['VICReg_backbone_resnet18', 'VICReg_backbone_resnet18_pretrained']